# Unlocking Langchain Framework

In [1]:
from RestaurantNameGenerator.secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

# LLM Practice

In [2]:
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0.7)
city_name = llm.predict("I want to open a Indian restaurant in UK, give me only the best city name for that.")
print(city_name)



Birmingham.


In [4]:
llm("I want to open a Turkish restaurant in US, give me only the best city name for that.")

'\n\nSan Francisco, California.'

# Prompt Templates

In [6]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, give me only one city name for that."
)
prompt = prompt_template_name.format(culture="Arabic", location="UK")
print("prompt:",prompt)
print(llm.predict(prompt))

prompt: I want to open a Arabic restaurant in UK, give me only one city name for that.


Birmingham


# Chains

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run(culture="Arabic", location="Turkey")

'\n\nIstanbul.'

In [8]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run(culture="Mexican", location="Italy")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a Mexican restaurant in Italy, give me only one city name for that.

> Finished chain.


'\n\nRome.'

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['location'],
    template = "I want to open a Turkish restaurant in {location}, give me only the best city name for that."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [12]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])

content = chain.run("Germany")
content

'\n\n-Hamburger\n-Cheeseburger\n-Veggie Burger\n-Fried Onion Rings\n-French Fries\n-Fried Fish Sandwich\n-Grilled Chicken Sandwich\n-Hot Dog\n-Bratwurst\n-Salad\n-Baked Potato\n-Onion Soup\n-Chili'

#### Sequential Chain

In [33]:
llm = OpenAI(temperature=0.7)

prompt_template_city_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, Suggest a fancy city for this."
)

city_name_chain = LLMChain(llm=llm, prompt=prompt_template_city_name, output_key="city_name")

In [34]:
llm = OpenAI(temperature=0.7)

prompt_template_res_name = PromptTemplate(
    input_variables = ['city_name'],
    template = "Suggest a restaurant name in {city_name}."
)

retaurant_name_chain = LLMChain(llm=llm, prompt=prompt_template_res_name, output_key="restaurant_name")

In [35]:
llm = OpenAI(temperature=0.7)

prompt_template_menu_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_menu_items, output_key="menu_items")

In [36]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [city_name_chain, retaurant_name_chain, food_items_chain],
    input_variables = ['culture', 'location'],
    output_variables = ['city_name', 'restaurant_name', "menu_items"]
)

In [37]:
chain({"culture": "Indian", "location":"Norway"})

{'culture': 'Indian',
 'location': 'Norway',
 'city_name': '\n\nBergen is a great choice for opening an Indian restaurant in Norway. With a population of over 270,000, it is the second largest city in Norway and is a popular tourist destination. It is known for its beautiful coastal scenery, its vibrant cultural life, and its thriving culinary scene. It has a vibrant international community, with a wide variety of ethnic restaurants, and would be a great place to open an Indian restaurant.',
 'restaurant_name': '\n\nWe suggest the name "Tandoori Temptations" for an Indian restaurant in Bergen, Norway.',
 'menu_items': '\n\nTandoori Temptations Menu\n\nAppetizers\n\n• Vegetable Samosas\n• Paneer Tikka\n• Tandoori Chicken\n• Onion Bhaji\n• Aloo Tikki\n\nMain Courses\n\n• Butter Chicken\n• Palak Paneer\n• Lamb Rogan Josh\n• Chana Masala\n• Biryani\n\nAccompaniments\n\n• Naan Bread\n• Raita\n• Fried Onion\n• Cucumber Salad\n• Mango Chutney\n\nDesserts\n\n• Gulab Jamun\n• Kulfi\n• Rasmalai\

## Agents

In [ ]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"